In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
class DenseConv(nn.Module):
    def __init__(self, input_channel_size, output_channel_size):
        super(DenseConv, self).__init__()
    self.conv1 = nn.conv1d(in_channels = input_channel_size, out_channels = output_channel_size, kernel_size = 1, stride =1)
    self.relu = nn.Relu()
    def forward(self, inputs):
        conv1d_outputs = self.relu(self.conv1(inputs))
        return conv1d_outputs
    
def causalconv1(nn.Module):
    def __init__(self, input_channel_size, output_channel_size, kernel_size, stride, dilation):
        super(causalconv1, self).__init__()
        self.padding = (kernel_size-1)*dilation
        self.dilationConv1d = nn.conv1d(in_channels = input_channel_size, out_channels = output_channel_size, \
                                    kernel_size = kernel_size, stride =stride, padding = self.padding, dilation= dilation)
    def forward(self, inputs):
        causal_conv1d_outputs = self.dilationConv1d(inputs)[:,:,:-self.padding[0]]
        return causal_conv1d_outputs
        
def temporalConv(nn.Module):
    def __init__(self, input_channel_size, output_channel_size, intermediate_channel_size, kernel_size, stride, dilation):
        super(causalconv1, self).__init__()
        self.conv_dense_pre = DenseConv(input_channel_size, output_channel_size)
        self.filter = causalconv1(output_channel_size, intermediate_channel_size, kernel_size, stride, dilation)  
        self.gate = causalconv1(output_channel_size, intermediate_channel_size, kernel_size, stride, dilation) 
        self.conv_dense_post = DenseConv(intermediate_channel_size, output_channel_size)
        self.tanh = nn.Tanh()
        self.relu = nn.Relu()
    def forward(self, inputs):
        outputs_pre = self.conv_dense_pre(inputs)
        outputs_filter = self.filter(outputs)
        outputs_gate = self.filter(outputs)
        outputs_post = self.tanh(outputs_filter) * self.tanh(outputs_gate)
        outputs_post = self.conv_dense_post(outputs_post)
        outputs = outputs_pre + outputs_post
        return outputs, outputs_post
        
def temporalConv(nn.Module):
    def __init__(self, input_channel_size, output_channel_size, \
                 intermediate_channel_size, post_channel_size,kernel_size, stride, dilation, device):
        super(causalconv1, self).__init__()
        ### N层temporalConv，需要将其size写清楚：
        self.tcn_list = nn.ModuleList([temporalConv(input_channel_size, output_channel_size, \
                                               intermediate_channel_size, kernel_size, stride, dilation)])
        for i in dilation_list:
            self.tcn_list.append(temporalConv(output_channel_size, output_channel_size, \
                                               intermediate_channel_size, kernel_size, stride, dilation))
        self.conv_post = nn.Sequential(DenseConv(output_channel_size, post_channel_size),
                                      nn.Dropout(0.2),
                                      DenseConv(post_channel_size,1))
            
    def forward_t(self,inputs):
        outputs_post_list = []
        for tcn in self.tcn_list:
            outputs, outputs_post = tcn(inputs)
            inputs = outputs
            outputs_post_list.append(outputs_post)
        sum_outputs_post = sum(outputs_post_list)
        outputs = self.conv_post(sum_outputs_post)
        return outputs    
        
    def forward(self, inputs, teacher_forcing_rate):
        if teacher_forcing_rate == 1:
            outputs = self.forward_t(inputs)
            return outputs[:, :-self.decoder_seq_len, :]
        
        else:
            batch_size = inputs.size(0)
            decoder_inputs = inputs[:, :, :-self.decoder_seq_len+1].clone()
            decoder_outputs = torch.zeros(batch_size, decoder_inputs, device = self.device)
            
            for i in range(self.decoder_seq_len):
                outputs = self.forward_t(decoder_inputs)[:,:,-1:] ### [129, 22, decoder_size]
                decoder_exog = inputs[:,1:,-self.decoder_seq_len+1+i:] ### [129, 22-1, decoder_size]
                decoder_exog = torch.cat([outputs,decoder_exog],dim=1)
                decoder_inputs = torch.cat([decoder_inputs[:,:,1:], decoder_exog], dim =2 )
            return decoder_outputs.unsqueeze(2)
                
                
                
            



In [4]:
nn.Conv1d?

Init signature: nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
Docstring:     
Applies a 1D convolution over an input signal composed of several input
planes.

In the simplest case, the output value of the layer with input size
:math:`(N, C_{in}, L)` and output :math:`(N, C_{out}, L_{out})` can be
precisely described as:

.. math::

    \begin{equation*}
    \text{out}(N_i, C_{out_j}) = \text{bias}(C_{out_j}) +
                            \sum_{k = 0}^{C_{in} - 1} \text{weight}(C_{out_j}, k) \star \text{input}(N_i, k)
    \end{equation*},

where :math:`\star` is the valid `cross-correlation`_ operator,
:math:`N` is a batch size, :math:`C` denotes a number of channels,
:math:`L` is a length of signal sequence.

* :attr:`stride` controls the stride for the cross-correlation, a single
  number or a one-element tuple.

* :attr:`padding` controls the amount of implicit zero-paddings on both sides
  for :attr:`padding` number of points

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np